Leacture name: Least square classifier from scratch
Lecture goals are:
- Learn least square regression form scrathc
- Optimization via normal eqn and gradient descent
- Prediction on new set of points
- encode categorical features

and here we'll see
- least square classification implementation
    - Training data
    - Model
    - Loss function
    - Normal equation
    - Iterative optimization(GD, MBGD and SGD)
    - inference
    - works for multi-class classification

we'll see four demos:
1) linearly separable binary classification
2) Linearly separable binary classification with outliers
3) multi-class setup
4) we'll perform lsc on nonlinearly separabel data where we have to use Polynomial least square classification

In [2]:
# we will use polynomial transformationcode from last week
import itertools
import functools

def get_combinations (x, degree):
    return itertools.combinations_with_replacement(x, degree)

def compute_new_features(items):
    # reduce (lambda x, y: x*y,[1,2,3,4,5]) calculates ((((1*2)*3)*4)*5)
    return functools.reduce(lambda x, y: x*y, items)

def polynomial_transform(x, degree, logging=False):
    # Converts to feature matrix
    if x.ndim == 1:
        x = x[:, None]
    x_t = x.transpose()
    features = [np.ones(len(x))] # Populates 1s as first feature for each example

    if logging:
        print("Input:", x)
    for degree in range(1,degree+1):
        for items in get_combinations(x_t, degree):
            features.append(compute_new_features(items))
            if logging:
                print(items, ":", compute_new_features(items))
    if logging:
        print(np.asarray(features).transpose())
    return np.asarray(features).transpose()

In [3]:
#In this class w have a function called LabelTansformer() which converts a discrete class into onehot encoding(From PRML librearY)
import numpy as np


class LabelTransformer(object):
    """
    Label encoder decoder
    Attributes
    ----------
    n_classes : int
        number of classes, K
    """

    def __init__(self, n_classes:int=None):
        self.n_classes = n_classes

    @property
    def n_classes(self):
        return self.__n_classes

    @n_classes.setter
    def n_classes(self, K):
        self.__n_classes = K
        self.__encoder = None if K is None else np.eye(K)

    @property
    def encoder(self):
        return self.__encoder

    def encode(self, class_indices:np.ndarray):
        """
        encode class index into one-of-k code
        Parameters
        ----------
        class_indices : (N,) np.ndarray
            non-negative class index
            elements must be integer in [0, n_classes)
        Returns
        -------
        (N, K) np.ndarray
            one-of-k encoding of input
        """
        if self.n_classes is None:
            self.n_classes = np.max(class_indices) + 1

        return self.encoder[class_indices]

    def decode(self, onehot:np.ndarray):
        """
        decode one-of-k code into class index
        Parameters
        ----------
        onehot : (N, K) np.ndarray
            one-of-k code
        Returns
        -------
        (N,) np.ndarray
            class index
        """

        return np.argmax(onehot, axis=1)

In [5]:
#lets try the above LabelTransform function
binary_labels = LabelTransformer(2).encode(np.array([1,0,1,0]))
binary_labels

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [1]:
import numpy as np


def create_toy_data(add_outliers=False, add_class=False):
    x0 = np.random.normal(size=50).reshape(-1,2)-1
    x1 = np.random.normal(size=50).reshape(-1,2)+1
    if add_outliers:
        x1 = np.random.normal(size=50).reshape9-1,2 + np.array([5.,10.])
        return np.concatenate([x0,x1,x_1]),np.concatenate([np.zeroes(25),np.ones(30)]).astype(n.int)
    if add_class:
        x2 = np.random.normal(size=50).reshape(-1,2)+2
        return np.concatenate([x0,x1]),np.concatenate([np.zeroes(25),np.ones(25)]).astype(np.int)

Lets difine a generic data preprocessing function that
- Generates synthetic data by calling "create_toy_data"
- performs polynomial transformation on featurte set
- divides the data into training and test set with "train_test_split" api with sklearn lib
- performs label transformation for both train and test sets

In [ ]:
##@title Data Preprocessing
from sklearn.model_selection import train_test_split
def preprocess(add_class=False, add_outliers=False, degree=1):
    x,y = create_toy_data(add_outliers,add_class)
    x_poly = polynomial_transform(x,degree=degree)
